### Homework 5: Question search engine

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to really solve this task using context-aware embeddings.

__Warning:__ this task assumes you have seen `seminar.ipynb`!

In [1]:
%pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 52.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 83.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 5.5 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.12.2-py3-none-any.whl size=1265684 sha256=dfeeb2d4f7373d4dea0af89862e03f21cd0e15be3f7b60e08a49292e687e3aee
  Stored in directory: /root/.cache/pip/wheels/06/c8/39/10f68166de0a2a12c511c3c0569128b62be534edc9a224782c
Successfully built deepspeed
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.

### Load data and model

In [2]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

Repo card metadata block was not found. Setting CardData to empty.
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]



Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [3]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

### Tokenize the data

In [4]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [5]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[101, 1731, 1110, 1103, 1297, 1104, 170, 12523, 2377, 136, 7426, 1128, 5594, 1240, 1319, 5758, 136,  ...


### Task 1: evaluation (1 points)

We randomly chose a model trained on QQP - but is it any good?

One way to measure this is with validation accuracy - which is what you will implement next.

Here's the interface to help you do that:

In [6]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=1, shuffle=False, collate_fn=transformers.default_data_collator
)

In [7]:
for batch in val_loader:
     break  # here be your training code
print("Sample batch:", batch)

with torch.no_grad():
  predicted = model(
      input_ids=batch['input_ids'],
      attention_mask=batch['attention_mask'],
      token_type_ids=batch['token_type_ids']
  )

print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

Sample batch: {'labels': tensor([0]), 'idx': tensor([0]), 'input_ids': tensor([[  101,  2009,  1132,  2170,   118,  4038,  1177,  2712,   136,   102,
          2009,  1132,  1117, 10224,  4724,  1177,  2712,   136,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,   

__Your task__ is to measure the validation accuracy of your model.
Doing so naively may take several hours. Please make sure you use the following optimizations:

- run the model on GPU with no_grad
- using batch size larger than 1
- use optimize data loader with num_workers > 1
- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [10]:

# <A whole lot of YOUR CODE HERE>
# ...


# accuracy = <Validation accuracy, between 0 and 1>
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

model.eval()

val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size=64,
    shuffle=False,
    collate_fn=transformers.default_data_collator,
    num_workers=2
)

correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            token_type_ids=batch['token_type_ids']
        )

        probabilities = torch.softmax(outputs.logits, dim=1)

        _, predicted_labels = torch.max(probabilities, dim=1)

        correct_predictions += (predicted_labels == batch['labels']).sum().item()
        total_predictions += batch['labels'].size(0)
accuracy = correct_predictions / total_predictions
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9084


In [11]:
assert 0.9 < accuracy < 0.91

### Task 2: train the model (5 points)

For this task, you have two options:

__Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.


__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

models = {
    "deberta":"Tomor0720/deberta-base-finetuned-qqp",
    "roberta": "JeremiahZ/roberta-base-qqp",
    "distilbert": "assemblyai/distilbert-base-uncased-qqp",
    "ernie":"rajiv003/ernie-finetuned-qqp",
    "albert": "Alireza1044/albert-base-v2-qqp"
}

for name, model_name in models.items():
    print(f'------{model_name}-------')
    newtokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

------Tomor0720/deberta-base-finetuned-qqp-------


------JeremiahZ/roberta-base-qqp-------


------assemblyai/distilbert-base-uncased-qqp-------


------rajiv003/ernie-finetuned-qqp-------


------Alireza1044/albert-base-v2-qqp-------


In [13]:
dataset = qqp['validation']

In [24]:
import os
import time
import pandas as pd
from tqdm.auto import tqdm
from transformers import pipeline

def evaluate_model(model, val_loader, device, newtokenizer):
    model.eval()
    correct_predictions = 0
    total_predictions = 0

    # Start timing
    start_time = time.time()

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Evaluating"):
              batch = {k: v.to(device) for k, v in batch.items()}
              inputs = {
                  "input_ids": batch["input_ids"],
                  "attention_mask": batch["attention_mask"]
              }
              if "token_type_ids" in newtokenizer.model_input_names:
                  inputs["token_type_ids"] = batch.get("token_type_ids", None)

              # Forward pass, get model outputs
              outputs = model(**inputs)
              probabilities = torch.softmax(outputs.logits, dim=1)

              _, predicted_labels = torch.max(probabilities, dim=1)

              correct_predictions += (predicted_labels == batch['labels']).sum().item()
              total_predictions += batch['labels'].size(0)

    # End timing
    end_time = time.time()

    # Calculate time elapsed and speed
    time_elapsed = end_time - start_time
    speed = total_predictions / time_elapsed

    return correct_predictions / total_predictions, speed


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

results = []

for name, model_name in models.items():

    newtokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

    def preprocess_function(examples):
        result = newtokenizer(
            examples['text1'], examples['text2'],
            padding='max_length', max_length=MAX_LENGTH, truncation=True
        )
        result['label'] = examples['label']
        return result
    
    data_val_preprocessed = dataset.map(preprocess_function, batched=True)

    val_loader = torch.utils.data.DataLoader(
      data_val_preprocessed,
      batch_size=64,
      shuffle=False,
      collate_fn=transformers.default_data_collator,
      num_workers=2
    )

    accuracy, speed = evaluate_model(model, val_loader, device, newtokenizer)
    
    print("Model",name,
        "Accuracy",accuracy,
        "Speed (samples/sec)",speed
         )
    
    results.append({
        "Model": name,
        "Accuracy": accuracy,
        "Speed (samples/sec)": speed
    })

df = pd.DataFrame(results)

print(df.to_markdown())

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Evaluating:   0%|          | 0/632 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Model deberta Accuracy 0.9127627999010636 Speed (samples/sec) 217.89080187552216


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Evaluating:   0%|          | 0/632 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Model albert Accuracy 0.9049715557754143 Speed (samples/sec) 236.5968158218924
|    | Model      |   Accuracy |   Speed (samples/sec) |
|---:|:-----------|-----------:|----------------------:|
|  0 | deberta    |   0.912763 |               217.891 |
|  1 | roberta    |   0.91531  |               270.533 |
|  2 | distilbert |   0.899258 |               527.038 |
|  3 | ernie      |   0.915211 |               268.591 |
|  4 | albert     |   0.904972 |               236.597 |


## Model Comparison for Duplicate Question Identification

### Accuracy
From the table, it is evident that **RoBERTa** and **ERNIE** models boast the highest accuracy, both exceeding 91.5%, with RoBERTa marginally leading. 
- RoBERTa: 91.531%
- ERNIE: 91.521%

### Speed
In terms of inference speed, **DistilBERT** outperforms the others by a significant margin which is nearly double the speed of DeBERTa and ERNIE, and about twice as fast as RoBERTa.

### Size
About model size, **ALBERT** stands out as the most compact model, occupying a mere 46.8 MB, which is substantially smaller compared to the others, with DistilBERT being the second smallest at 268 MB. DeBERTa is the largest model, requiring 557 MB of storage space.

### Summary
- For **high accuracy** needs: Choose **RoBERTa** or **ERNIE**.
- For a balance between **speed and accuracy**: Opt for **DistilBERT**.
- For **resource-constrained environments**: Go with **ALBERT**, accepting a minor compromise on accuracy.



|    | Model      |   Accuracy |   Speed (samples/sec) |    Size(MB) |
|---:|:-----------|-----------:|----------------------:|------------:|
|  0 | deberta    |   0.912763 |               217.891 |         557 |
|  1 | roberta    |   0.91531  |               270.533 |         499 |
|  2 | distilbert |   0.899258 |               527.038 |         268 |
|  3 | ernie      |   0.915211 |               268.591 |         438 |
|  4 | albert     |   0.904972 |               236.597 |        46.8 |

### Task 3: try the full pipeline (2 points)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 5 examples.

## We use finetuned debertav3 model

In [11]:
model_name = "Tomor0720/deberta-base-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## We use **Validation Set** here for testing the naive method

In [50]:
dataset = qqp['validation']

## Naive Method

In [51]:
input_question = "Is Python the best language?"
query = input_question

def tokenize_with_query(examples):
    return tokenizer(
        examples['text1'], [query] * len(examples['text1']),
        padding='max_length', max_length=MAX_LENGTH, truncation=True,
        return_tensors='pt'
    )

dataset_with_query = dataset.map(tokenize_with_query, batched=True)

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

In [58]:
from tqdm.auto import tqdm

def inference_text(model, tokenizer, query, dataset, batch_size=64,topk=5):
    model.eval()

    train_loader = DataLoader(
        dataset, 
        batch_size=batch_size,
        collate_fn=transformers.default_data_collator,
        shuffle=False  
    )

    results = []
    with torch.no_grad():
        for batch in tqdm(train_loader):
            batch = {k: v.to(device) for k, v in batch.items()}
            indices = batch.pop('idx').tolist()   
            outputs = model(**batch)   

            logits = outputs.logits
            probs = torch.softmax(logits, dim=1)
            scores = probs[:, 1]  
            results.extend(zip(indices, scores.tolist()))

    results.sort(key=lambda x: x[1], reverse=True)
    top_results = results[:topk]
    top_duplicates = [(dataset[idx]['text1'],score) for idx, score in top_results]
    return top_duplicates

top_duplicates = inference_text(model, tokenizer, input_question, dataset_with_query)
for text, score in top_duplicates:
    print(f"Score: {score:.4f}, Text: {text}")

  0%|          | 0/632 [00:00<?, ?it/s]

Score: 0.9977, Text: If more vacuum energy appears with expansion and it has no limit, can infinite of this energy be created? If yes is energy infinite?
Score: 0.9971, Text: If universe expands and vacuum energy is created with it (with no limit),is there infinite potential energy/infinite vacuum energy that can be created?
Score: 0.9898, Text: *1-(800) 439–2178* BITDEFENDER Antivirus Tech Support Number, BITDEFENDER Antivirus Support Phone Number?
Score: 0.9850, Text: If you are traveling in a car going 60mph with a fly inside the car, is the fly traveling 60mph?
Score: 0.9850, Text: If you are traveling in a car going 60mph with a fly inside the car, is the fly traveling 60mph?


## Optimized Method
### Step 1: Create a TF-IDF Vectorizer to Shortlist Candidates
### Step 2: Define a Function to Get Top-N Similar Questions Using Cosine Similarity
### Step 3: Use the Shortlist in the Transformer Model



In [138]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def preprocess_for_tfidf(texts):
    return [text.lower() for text in texts]

all_questions = preprocess_for_tfidf([example['text1'] for example in dataset])

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_questions)

In [139]:
def get_top_n_similar_questions(query, tfidf_matrix, tfidf_vectorizer, top_n=100):
    query_vector = tfidf_vectorizer.transform([query.lower()])
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_n_indices = np.argsort(cosine_similarities)[-top_n:]

    return top_n_indices, cosine_similarities[top_n_indices]


In [147]:
def inference_text_optimized(model, tokenizer, query, dataset, device, tfidf_matrix, tfidf_vectorizer, topk=5, prefilter_top_n=100):
    top_n_indices, _ = get_top_n_similar_questions(query, tfidf_matrix, tfidf_vectorizer, top_n=prefilter_top_n)
    shortlisted_dataset = dataset.select(top_n_indices)
    dataset_with_query = shortlisted_dataset.map(tokenize_with_query, batched=True)
    return inference_text(model, tokenizer, query, dataset_with_query, batch_size=64, topk=topk)


In [148]:
top_duplicates_optimized = inference_text_optimized(model, tokenizer, input_question, dataset, device, tfidf_matrix, tfidf_vectorizer)
for text, score in top_duplicates_optimized:
    print(f"Score: {score:.4f}, Text: {text}")


  0%|          | 0/2 [00:00<?, ?it/s]

Score: 0.3511, Text: Is Python better than Java? Why or why not?
Score: 0.0326, Text: What about c language?
Score: 0.0303, Text: What's the difference between a programming language and a scripting language?
Score: 0.0125, Text: Why should I learn Python instead of Java?
Score: 0.0077, Text: Which is the best source to learn python?


### We can see for the same query it is way more faster than the naive method
### Next we do for 5 example questions:


In [149]:
dataset = qqp['train']

In [150]:
def preprocess_for_tfidf(texts):
    return [text.lower() for text in texts]

all_questions = preprocess_for_tfidf([example['text1'] for example in dataset])

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_questions)

In [151]:
input_questions = [
    "Is Python the best language for data science?",
    "What is the best way to chill on the weekends?",
    "Why we need to study?",
    "What is the best stock",
    "Where we can find the datasets?"
]

In [152]:
for input_question in input_questions:
    print(f"Input Question: {input_question}\n")
    top_duplicates_optimized = inference_text_optimized(
        model,
        tokenizer,
        input_question,
        dataset,  
        device,
        tfidf_matrix,
        tfidf_vectorizer
    )
    for text, score in top_duplicates_optimized:
        print(f"Score: {score:.4f}, Text: {text}")
    print("\n" + "-"*80 + "\n")

Input Question: Is Python the best language for data science?



Map:   0%|          | 0/100 [00:00<?, ? examples/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Score: 0.9540, Text: Is Python a good first programming language to learn?
Score: 0.9125, Text: Is Python really the best programming language for beginners?
Score: 0.1195, Text: Which is the best book/resource to learn Python programming language?
Score: 0.1089, Text: Should Python be my first programming language?
Score: 0.0428, Text: Which is the best programming language to do my project, Python or Java?

--------------------------------------------------------------------------------

Input Question: What is the best way to chill in the weekends?



Map:   0%|          | 0/100 [00:00<?, ? examples/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Score: 0.0003, Text: What is a stock market? What actually takes place in a stock market?
Score: 0.0003, Text: What is a stock market? What actually takes place in a stock market?
Score: 0.0001, Text: What is the best stock picking software?
Score: 0.0001, Text: Which is the best Indian stock market app?
Score: 0.0001, Text: Which is the best Indian stock market app?

--------------------------------------------------------------------------------

Input Question: Where we can find the datasets?



Map:   0%|          | 0/100 [00:00<?, ? examples/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Score: 0.0001, Text: Where Can I find the best seo company?
Score: 0.0001, Text: What are the best pop and where can I find it?
Score: 0.0001, Text: Which is the best database to store and analyze big datasets in my personal computer?
Score: 0.0001, Text: What is love? How can we find that we are in love?
Score: 0.0001, Text: What is love? How can we find that we are in love?

--------------------------------------------------------------------------------



__Bonus:__ for bonus points, try to find a way to run the function faster than just passing over all questions in a loop. For isntance, you can form a short-list of potential candidates using a cheaper method, and then run your tranformer on that short list. If you opted for this solution, please keep both the original implementation and the optimized one - and explain briefly what is the difference there.